### Dodanie atrybutów ze spisu powszechnego 2011 do obiektów zabudowy urban-atlas 2012

[Spis powszechny:](https://geo.stat.gov.pl/aktualnosci)    (aktualności z datą 12.01.2017)

[Urban atlas:](http://land.copernicus.eu/local/urban-atlas)

Program poniżej dodaje dla każdego obiektu dowolnej polskiej warstwy pokrycia urban-atlas dane o ludności pozyskane w spisie powszechnym 2011 i udostępnione w siatce kilometrowej.

Do programu (ArcGIS Pro) wprowadzamy pustą geobazę, SHP urban-atlas dla danego obszaru i część siatki ze spisu powszechnego pokrywająca ten obszar. Obie warstwy powinny być sprowadzone do jednego układu współrzędnych (np. ETRS 1989 Poland CS92).

In [3]:
import numpy as np
import arcpy

arcpy.env.workspace=r"C:\SP8\POPULACJA\LudCorUrb\LudCorUrb.gdb"


corine_urb=r"C:\SP8\POPULACJA\CORINE_URBAN_POMORSKIE.shp"                         # ETRS 1989 Poland CS92 corine_urb 
geostat_pop=r"C:\SP8\POPULACJA\PD_STAT_GRID_CELL_2011_URB_POM.shp"  # ETRS 1989 Poland CS92   siatka pokrywająca corine_urb
corine_urb2=r"C:\SP8\POPULACJA\CORINE_URBAN_POMORSKIE_POP2.shp" 

# do dopasowania z powyższych (pole z ID w warstwie po Union)
fid1="FID_PD_STAT_GRID_CELL_2011_URB_POM"
fid2="FID_CORINE_URBAN_POMORSKIE"

In [4]:
# Krok1 
arcpy.Union_analysis([corine_urb, geostat_pop], "temp_1")
# Krok2 - tylko pokrycia zamieszkane
where_clause = "CODE2012='11100' OR CODE2012='11210' OR CODE2012='11220' OR CODE2012='11230' OR CODE2012='11240' OR CODE2012='11300'"
arcpy.Select_analysis("temp_1","temp_2", where_clause)
# Krok 3  - sortowanie po ID kwadratów statystycznych
arcpy.Sort_management("temp_2", "temp_2s", [[fid1, "ASCENDING"]])
# Krok 4 
rekordy=int(arcpy.GetCount_management("temp_2s").getOutput(0))
print('rekordy',rekordy)

rekordy 24194


In [5]:
# Struktury danych

tab1=np.zeros((rekordy,3),int)       #  0-ID kwadratu statystycznego, 1-ID pokrycia Corine, 2-kod pokrycia
                                     
tab2=np.zeros((rekordy,16),float)    # 0-TOT, 1-TOT_0_14, 2-TOT_15_64, 3-TOT_65__, 4-TOT_MALE, 5-TOT_FEM,
                                    # 6- MALE_0_14, 7-MALE_15_64,8-MALE_65__,9-FEM_0_14, 10-FEM_15_64, 11-FEM_65__,
                                    # 12-Shape_Area, 13-procent pokrycia pop, 14 - pow*proc, 15-suma pow*proc dla ID kw.stat
        
tab3=np.zeros((rekordy,12),float)   # Wyniki dla rekordu: 0-TOT, 1-TOT_0_14, 2-TOT_15_64, 3-TOT_65__, 4-TOT_MALE, 5-TOT_FEM,
                                    # 6- MALE_0_14, 7-MALE_15_64,8-MALE_65__,9-FEM_0_14, 10-FEM_15_64, 11-FEM_65__,

# do tablic tab1, tab2
with arcpy.da.SearchCursor("temp_2s",[fid1,fid2,"CODE2012","TOT","TOT_0_14",
                                     "TOT_15_64","TOT_65__","TOT_MALE",
                                     "TOT_FEM","MALE_0_14","MALE_15_64",
                                      "MALE_65__","FEM_0_14","FEM_15_64",
                                      "FEM_65__","Shape_Area"]) as cur:
    ii=0
    for row in cur:
        tab1[ii,0]=row[0];tab1[ii,1]=row[1];tab1[ii,2]=int(row[2])
        tab2[ii,0]=row[3];tab2[ii,1]=row[4];tab2[ii,2]=row[5]
        tab2[ii,3]=row[6];tab2[ii,4]=row[7];tab2[ii,5]=row[8]
        tab2[ii,6]=row[9];tab2[ii,7]=row[10];tab2[ii,8]=row[11]
        tab2[ii,9]=row[12];tab2[ii,10]=row[13];tab2[ii,11]=row[14]
        tab2[ii,12]=row[15]
        ii+=1
del cur

In [6]:
# OPCJONALNIE kontrola procesu
for ii in range(0,10):
    print(ii,tab1[ii,0],tab1[ii,1],tab1[ii,2],round(tab2[ii,12],1),tab2[ii,13],round(tab2[ii,14],2),round(tab2[ii,15]))   

0 31 17155 11300 4990.4 0.0 0.0 0.0
1 32 16357 11300 5015.9 0.0 0.0 0.0
2 32 16292 11300 3985.4 0.0 0.0 0.0
3 32 16360 11300 35.0 0.0 0.0 0.0
4 32 16320 11300 5636.4 0.0 0.0 0.0
5 32 16344 11300 3858.7 0.0 0.0 0.0
6 32 16338 11300 6379.1 0.0 0.0 0.0
7 40 18235 11300 2686.0 0.0 0.0 0.0
8 40 18177 11300 2652.1 0.0 0.0 0.0
9 42 18201 11300 2725.1 0.0 0.0 0.0


In [7]:
# Wpisanie procentowego pokrycia populcja
# tab2[,13 - Proc], tab2[,14 - Pow*Proc]

def proc(kodd):
    if kodd==11100:
        pp=0.90
    elif kodd==11210:
        pp=0.65
    elif kodd==11220:
        pp=0.4
    elif kodd==11230:
        pp=0.2
    elif kodd==11240:
        pp=0.05
    elif kodd==11300:
        pp=0.5
    return pp

for i in range(0,rekordy):
    tab2[i,13]=proc(tab1[i,2])
    tab2[i,14]=tab2[i,13]*tab2[i,12]

![Obrazek 0](img\alg2.png)

In [8]:
# Obliczanie sumy pow*proc dla danego fid1

tab_x=np.zeros((rekordy),float)   # Suma pow * proc dla danego fid1-indeksu w tablicy)

iis=tab1[0,0]  # indeks fid1  
suma=tab2[0,14] #pow*proc

for i in range(1,rekordy):
                if  tab1[i,0]==iis:
                      suma=suma+tab2[i,14]  # obliczanie sumy pow*proc                      
                else:
                    tab_x[iis]=suma
                    iis=tab1[i,0];suma=tab2[i,14]
tab_x[iis]=suma               

In [9]:
# OPCJONALNIE  kontrola procesu

for i in range(31,42):
    print(i,tab_x[i])

31 2495.2248396265577
32 12455.229351912107
33 0.0
34 0.0
35 0.0
36 0.0
37 0.0
38 0.0
39 0.0
40 2669.035449222048
41 0.0


In [12]:
# wpisanie sum pow*proc 
for i in range(0,rekordy):
    tab2[i,15]=tab_x[tab1[i,0]]

In [13]:
# obliczenie populacji
for i in range(0,rekordy):
    for k in range(0,12):
        tab3[i,k]=(tab2[i,k]/tab2[i,15])*tab2[i,14]

In [14]:
# OPCJONALNIE kontrola procesu
for i in range(0,10):
    print(tab1[i,1],round(tab3[i,0],1),round(tab3[i,1],1),round(tab3[i,2],1),round(tab3[i,3],1),
    round(tab3[i,4],1),round(tab3[i,5],1),round(tab3[i,6],1),round(tab3[i,7],1),round(tab3[i,8],1),
    round(tab3[i,9],1),round(tab3[i,10],1),round(tab3[i,11],1))

17155 38.0 9.0 25.0 4.0 21.0 17.0 4.0 15.0 0.0 5.0 10.0 3.0
16357 3.8 0.8 2.4 0.6 1.8 2.0 0.6 1.0 0.0 0.6 1.4 0.6
16292 3.0 0.6 1.9 0.5 1.4 1.6 0.5 0.8 0.0 0.5 1.1 0.5
16360 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
16320 4.3 0.9 2.7 0.7 2.0 2.3 0.7 1.1 0.0 0.7 1.6 0.7
16344 2.9 0.6 1.9 0.5 1.4 1.5 0.5 0.8 0.0 0.5 1.1 0.5
16338 4.9 1.0 3.1 0.8 2.3 2.6 0.8 1.3 0.0 0.8 1.8 0.8
18235 10.1 2.0 7.0 0.0 6.5 3.5 1.5 5.0 0.0 0.0 2.0 1.5
18177 9.9 2.0 7.0 0.0 6.5 3.5 1.5 5.0 0.0 0.0 2.0 1.5
18201 6.0 0.0 5.0 0.0 4.0 0.0 0.0 3.0 0.0 0.0 0.0 0.0


In [15]:
#0 - CORID,  1-TOT, 2-TOT_0_14, 2-TOT_15_64, 3-TOT_65__, 4-TOT_MALE, 5-TOT_FEM,
# 6- MALE_0_14, 7-MALE_15_64,8-MALE_65__,9-FEM_0_14, 10-FEM_15_64, 11-FEM_65__,
tabb="temptab1"
arcpy.CreateTable_management(arcpy.env.workspace,tabb)
arcpy.AddField_management(tabb,"CORID","LONG")
arcpy.AddField_management(tabb,"TOT","FLOAT")
arcpy.AddField_management(tabb,"TOT_0_14","FLOAT")
arcpy.AddField_management(tabb,"TOT_15_64","FLOAT")
arcpy.AddField_management(tabb,"TOT_65_","FLOAT")
arcpy.AddField_management(tabb,"TOT_MALE","FLOAT")
arcpy.AddField_management(tabb,"TOT_FEM","FLOAT")
arcpy.AddField_management(tabb,"MALE_0_14","FLOAT")
arcpy.AddField_management(tabb,"MALE_15_64","FLOAT")
arcpy.AddField_management(tabb,"MALE_65_","FLOAT")
arcpy.AddField_management(tabb,"FEM_0_14","FLOAT")
arcpy.AddField_management(tabb,"FEM_15_64","FLOAT")
arcpy.AddField_management(tabb,"FEM_65_","FLOAT")

<Result 'C:\\SP8\\POPULACJA\\LudCorUrb\\LudCorUrb.gdb\\temptab1'>

In [16]:
# wypełnienie tablicy

with arcpy.da.InsertCursor(tabb,["CORID","TOT","TOT_0_14","TOT_15_64","TOT_65_","TOT_MALE","TOT_FEM",
                                "MALE_0_14","MALE_15_64","MALE_65_","FEM_0_14","FEM_15_64","FEM_65_"]) as cur:
   for i in range(0,rekordy):
       rr=(tab1[i,1],round(tab3[i,0],2),round(tab3[i,1],2),round(tab3[i,2],2),round(tab3[i,3],2),round(tab3[i,4],2),
          round(tab3[i,5],2),round(tab3[i,6],2),round(tab3[i,7],2),round(tab3[i,8],2),round(tab3[i,9],2),
          round(tab3[i,10],2),round(tab3[i,11],2))
       cur.insertRow(rr)

del cur

In [17]:
# suma dla każdego CORID
LL=[["TOT","SUM"],["TOT_0_14","SUM"],["TOT_15_64","SUM"],["TOT_65_","SUM"],["TOT_MALE","SUM"],["TOT_FEM","SUM"],
    ["MALE_0_14","SUM"],["MALE_15_64","SUM"],["MALE_65_","SUM"],["FEM_0_14","SUM"],["FEM_15_64","SUM"],["FEM_65_","SUM"]]
arcpy.Statistics_analysis(tabb,"temptab2", LL, "CORID")

<Result 'C:\\SP8\\POPULACJA\\LudCorUrb\\LudCorUrb.gdb\\temptab2'>

In [18]:
arcpy.Select_analysis(corine_urb,corine_urb2)
LL2=["SUM_TOT","SUM_TOT_0_14","SUM_TOT_15_64","SUM_TOT_65_","SUM_TOT_MALE","SUM_TOT_FEM",
     "SUM_MALE_0_14","SUM_MALE_15_64","SUM_MALE_65_","SUM_FEM_0_14","SUM_FEM_15_64","SUM_FEM_65_"]
arcpy.JoinField_management(corine_urb2,'FID',"temptab2",'CORID', LL2)

<Result 'C:\\SP8\\POPULACJA\\CORINE_URBAN_POMORSKIE_POP2.shp'>

In [19]:
# Czyszcenie warstw tymczasowych w geobazie

arcpy.Delete_management('temp_1')
arcpy.Delete_management('temp_2')
arcpy.Delete_management('temp_2s')
arcpy.Delete_management('temptab1')
arcpy.Delete_management('temptab2')

<Result 'true'>

#### Przykładowa mapa wynikowa przedstawia procentowy udział mieszkańców o wieku  65 lat i więcej


![Obrazek 1](img\mieszkancy_64.jpg)